<a href="https://colab.research.google.com/github/mehrnaz1368/classification/blob/main/recommendation_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
import numpy as np
from math import sqrt
import matplotlib.pyplot as plt

In [17]:
movies_df= pd.read_csv('movies.csv')
ratings_df= pd.read_csv('ratings.csv')

In [18]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


# we want to move (year) from title column to independent column

In [19]:
movies_df['year']= movies_df.title.str.extract('(\(\d\d\d\d\))', expand=False)

#Remove the parentheses

In [20]:
movies_df['year']= movies_df.year.str.extract('(\d\d\d\d)', expand=False)

#remove the year from tiltle column

In [21]:
movies_df['title']= movies_df.title.str.replace('(\(\d\d\d\d\))', '')
movies_df['title']= movies_df['title'].apply(lambda x: x.strip())

<ipython-input-21-6735a105e9a6>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_df['title']= movies_df.title.str.replace('(\(\d\d\d\d\))', '')


In [22]:
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


#drop the genres

In [23]:
movies_df = movies_df.drop('genres', 1)

<ipython-input-23-cec6dd96bd4d>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  movies_df = movies_df.drop('genres', 1)


In [26]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


#remove timestamp from ratings_df

In [27]:
ratings_df=ratings_df.drop('timestamp', 1)

<ipython-input-27-9b66ec7e4abf>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  ratings_df=ratings_df.drop('timestamp', 1)


# Euclidean distance measurement
## relate every title to rate

---



In [47]:
userInput= [
    {'title': 'Breakfast Club, The', 'rating':5},
    {'title': 'Toy Story', 'rating':3.5},
    {'title': 'Jumanji', 'rating':2},
    {'title': "Puple Fiction", 'rating':5},
    {'title': 'Akira', 'rating':4.5},
    ]
inputMovies= pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Puple Fiction,5.0
4,Akira,4.5


#filter the movie by title

In [52]:
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
inputMovies=pd.merge(inputId, inputMovies)
inputMovies= inputMovies.drop('year', 1)
inputMovies

<ipython-input-52-1374d88dfdf2>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  inputMovies= inputMovies.drop('year', 1)


,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,1274,Akira,4.5
3,1968,"Breakfast Club, The",5.0


#We want to isolate users who watched a movie with the person we followed